In [146]:
# use pip to install all the libraries we need
import sys
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn seaborn | grep -v 'already satisfied'

# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import random

from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore') # remove warnings

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/trf1/.local/lib/python3.9/site-packages
sysconfig: /home/trf1/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [147]:
# read the data, dropping the ID column, as this is duplicated by pandas
raw_training_data = pd.read_csv(r"train.csv").drop(columns=["id"])
raw_test_data = pd.read_csv(r"test.csv").drop(columns=["id"])

copy_raw_training_data = raw_training_data.copy()  # copy the raw training data

In [148]:
display(copy_raw_training_data)

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1166,D-penicillamine,16839,F,N,N,N,N,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,1492,Placebo,17031,F,N,Y,N,N,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,1576,D-penicillamine,25873,F,N,N,Y,S,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,3584,D-penicillamine,22960,M,N,Y,N,N,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


Because many of the imputation methods only work on numerical data, we need to encode the categorical variables in our dataframe in some way. We chose to encode them with an integer for each category, since they all had very few options.

In [149]:
# specify how to encode each categorical variable as an integer
numerical_encode = {
    "Drug": {"Placebo": 0, "D-penicillamine": 1},
    "Sex": {"F": 0, "M": 1},
    "Ascites": {"N": 0, "Y": 1},
    "Hepatomegaly": {"N": 0, "Y": 1},
    "Spiders": {"N": 0, "Y": 1},
    "Edema": {"N": 0, "Y": 1, "S": 2},
}

In [150]:
# replace entries in categorical columns with specified numbers
training_data_encoded = copy_raw_training_data.replace(numerical_encode)
display(training_data_encoded)

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,1,21532,1,0,0,0,0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,0,19237,0,0,0,0,0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,0,13727,0,0,1,1,1,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,0,18460,0,0,0,0,0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,0,16658,0,0,1,0,0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1166,1,16839,0,0,0,0,0,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,1492,0,17031,0,0,1,0,0,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,1576,1,25873,0,0,0,1,2,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,3584,1,22960,1,0,1,0,0,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


In [151]:
# separate out the independent variables from the target
X_copy = training_data_encoded.drop(labels=['Status'], axis=1)
Y_copy = training_data_encoded["Status"]

In [152]:
def add_missing_values(X_full, Y_full, missing_rate):
    """
    Takes a dataframe and adds in some missing values.

    Parameters:
    - X_full: a dataframe containing all columns with independent variables
    - Y_full: a dataframe containing the response variable column only
    - missing_rate: a float between 0 and 1 which specifies the proportion of lines which should have missing values

    Returns:
    - X_missing: the dataframe X_full but with some missing values
    - Y_missing: an exact copy of Y_full
    """

    # copy the input dataframes
    X_missing = X_full.copy()
    y_missing = Y_full.copy()

    # remove a percentage of entries in each column at random, specified by missing_rate
    for col in X_missing.columns:
        X_missing.loc[X_missing.sample(frac=missing_rate).index, col] = np.nan

    return X_missing, y_missing

In [153]:
# apply the add_missing_values function to our data
X_copy_miss, Y_copy_miss = add_missing_values(X_copy, Y_copy, 0.5)
display(X_copy_miss)

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.3,NaN,NaN,172.0,1601.0,179.80,NaN,394.0,NaN,NaN
1,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,364.0,NaN,NaN,NaN,NaN,NaN,361.0,NaN,3.0
2,NaN,NaN,NaN,0.0,0.0,NaN,1.0,1.0,3.3,299.0,3.55,NaN,1029.0,NaN,NaN,199.0,11.7,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,256.0,NaN,58.0,NaN,71.30,96.0,269.0,10.7,3.0
4,788.0,NaN,16658.0,0.0,0.0,1.0,0.0,NaN,1.1,346.0,3.65,NaN,NaN,NaN,96.0,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,NaN,1.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,3.56,NaN,1629.0,NaN,NaN,344.0,NaN,NaN
7901,1492.0,NaN,17031.0,NaN,NaN,1.0,NaN,NaN,0.9,260.0,3.43,62.0,NaN,142.00,NaN,277.0,NaN,4.0
7902,1576.0,NaN,25873.0,0.0,NaN,0.0,1.0,NaN,NaN,NaN,3.19,51.0,NaN,69.75,62.0,200.0,NaN,NaN
7903,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,248.0,2.75,32.0,1003.0,57.35,NaN,NaN,10.6,NaN


In [154]:
# check that the correct proportions of values have been removed
X_copy_miss.isnull().sum() / len(X_copy_miss)

N_Days           0.499937
Drug             0.499937
Age              0.499937
Sex              0.499937
Ascites          0.499937
Hepatomegaly     0.499937
Spiders          0.499937
Edema            0.499937
Bilirubin        0.499937
Cholesterol      0.499937
Albumin          0.499937
Copper           0.499937
Alk_Phos         0.499937
SGOT             0.499937
Tryglicerides    0.499937
Platelets        0.499937
Prothrombin      0.499937
Stage            0.499937
dtype: float64

In [168]:
# Imputation 2: impute each missing value with the mean value from k nearest neighbours

imputer2 = KNNImputer(n_neighbors=2, weights="uniform")    # create a KNN imputer
results_X = pd.DataFrame(imputer2.fit_transform(X=X_copy_miss, y=Y_copy_miss), columns=X_copy_miss.columns) # apply the imputer to the data with missing values

# add the target back to the result
results = pd.concat([results_X, Y_copy_miss], axis=1)

display(results)

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,538.0,1.0,18142.0,0.5,0.0,0.0,0.0,0.0,2.30,307.0,3.75,172.0,1601.0,179.800,91.5,394.0,10.35,3.0,D
1,1704.5,0.0,16148.0,0.0,0.0,0.0,1.0,0.0,0.85,364.0,3.56,24.0,931.5,78.595,175.5,361.0,10.80,3.0,C
2,2502.5,0.5,16765.0,0.0,0.0,1.0,1.0,1.0,3.30,299.0,3.55,48.0,1029.0,172.050,135.0,199.0,11.70,3.5,D
3,3310.5,0.5,17972.5,0.0,0.0,0.5,0.5,0.0,1.35,256.0,3.50,58.0,1679.5,71.300,96.0,269.0,10.70,3.0,C
4,788.0,0.5,16658.0,0.0,0.0,1.0,0.0,0.0,1.10,346.0,3.65,37.5,734.5,60.450,96.0,261.0,10.95,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1717.0,1.0,15353.5,0.0,0.0,0.0,0.0,0.0,0.70,265.0,3.56,40.0,1629.0,87.575,98.0,344.0,10.95,2.0,C
7901,1492.0,0.5,17031.0,0.0,0.0,1.0,0.5,0.5,0.90,260.0,3.43,62.0,877.0,142.000,171.5,277.0,11.00,4.0,C
7902,1576.0,0.5,25873.0,0.0,0.0,0.0,1.0,0.0,2.90,162.5,3.19,51.0,986.5,69.750,62.0,200.0,10.70,2.5,D
7903,1122.5,1.0,18393.0,0.0,0.0,0.0,0.5,0.0,0.70,248.0,2.75,32.0,1003.0,57.350,114.5,246.5,10.60,3.0,D


In [ ]:

random.seed(25) # set a random seed
copy_raw_training_data = raw_training_data.copy()  # copy the raw training data
shape = copy_raw_training_data.shape  # store dimension of raw training data
num_entries = shape[0] * (shape[1] - 1)  # store number of entries for all independent variables
num_null = int(num_entries * 0.01)  # make 1% of the data be null entries

# randomly select entries and replace them with NaN, excluding the response variable
for _ in range(num_null):
    rand_row, rand_col = random.randint(0, shape[0] - 1) , random.randint(0, shape[1] - 2) # select a random row and column
    copy_raw_training_data.iloc[rand_row, rand_col] = np.nan # store entry as nan
    
missing_raw_training_data = copy_raw_training_data.copy()
    
# create a list to store all of the entries which are null: (row, column)
null_entries = [(row_index, col_index) 
                for row_index, row in enumerate(copy_raw_training_data.values) 
                for col_index, val in enumerate(row) 
                if pd.isnull(val)]

# store all of the entries for the original data set in the positions of the removed entries
actual_entries_store = {} # create dictionary for storing the actual values in the locations that are removed
for row, col in null_entries: # loop over every removed entry:
    col_name = copy_raw_training_data.columns[col] # store column name of current null entry
    if col_name not in actual_entries_store: # check if column name is already in the dictionary
        actual_entries_store[col_name] = [] # if not already in dictionary, create empty list
    actual_entries_store[col_name].append(raw_training_data.iloc[row, col])# add actual value to the list for correct column
    
# store the names of all categorical and numerical columns (excluding the response variable)
categorical_cols = [col for col in copy_raw_training_data.select_dtypes(include=['object']).columns.tolist() if col != 'Status']
num_cols = [x for x in copy_raw_training_data.columns.drop(['Status']) if x not in categorical_cols]
#--------------------------------------------------------------------------------------------------------------------------
# function to calculate performance metrics for numerical variables
def calc_num_metrics(actual_entries_store, imputed_entries_store, num_cols):
    """
    Calculates performance metric RMSE for numerical variables for the imputation method
    
    Parameters:
    - actual_entries_store: dictionary containing all entries that were set to NaN, keys are the data frames columns
    - imputed_entries_store: dictionary containing all imputed entries, indexed the same as actual_entries_store
    - num_cols: list of the numerical columns in the data frame
    
    Returns:
    - Data frame as a string showing the RMSE for the imputation method for each numerical variable
    """
    numerical_data = []
    for col_name, actual_vals in actual_entries_store.items(): # loop over all variables and actual values
        if col_name in num_cols: # if the current column is numerical:
            # change imputed and actual values to be numeric
            imputed_vals = pd.to_numeric(imputed_entries_store.get(col_name, []))
            actual_vals = pd.to_numeric(actual_vals)
            numerical_data.append({"Variable": col_name, "RMSE": round(mean_squared_error(actual_vals, imputed_vals, squared=False),3)}) # add the column name and its RMSE
    return pd.DataFrame(numerical_data).to_string(index=False)

# function to calculate performance metrics for categorical variables
def calc_categorical_metrics(actual_entries_store, imputed_entries_store, categorical_cols):
    """
    Calculates performance metric accuracy and F1 score for categorical variables for the imputation method
    
    Parameters:
    - actual_entries_store: dictionary containing all entries that were set to NaN, keys are the data frames columns
    - imputed_entries_store: dictionary containing all imputed entries, indexed the same as actual_entries_store
    - categorical_cols: list of the categorical columns in the data frame
    
    Returns:
    - Data frame as a string showing the accuracy and F1 score for the imputation method for each categorical variable
    """
    categorical_data = []
    for col_name in categorical_cols: # loop over all categorical variables
        # change imputed and actual values to be a string
        imputed_vals = [str(val) for val in imputed_entries_store.get(col_name, [])]
        actual_vals = [str(val) for val in actual_entries_store[col_name]]
        accuracy = accuracy_score(actual_vals, imputed_vals) # calculate accuracy score
        f1 = f1_score(actual_vals, imputed_vals, average='weighted') # calculate F1 score
        categorical_data.append({"Variable": col_name, "Accuracy": round(accuracy,3), "F1 Score": round(f1,3)}) # add the column name and its accuracy and F1 score
    return pd.DataFrame(categorical_data).to_string(index=False)

In [ ]:
# Imputation 1: impute missing values with median for numerical variables and mode for categorical variables

imputed1_entries_store = {} # create empty dictionary
for col_name in copy_raw_training_data.columns.drop(['Status']): # loop over each column of the data set 
    col_data = copy_raw_training_data[col_name] # store data for current column
    if col_data.dtype in ['float64', 'int64']: # check if data type of current column is numeric
        col_data.fillna(col_data.median(), inplace=True) # impute numerical nulls with median
    else:
        col_data.fillna(col_data.mode().iloc[0], inplace=True) # impute categorical nulls with mode
    imputed1_entries_store[col_name] = [col_data.loc[row] for row, col in null_entries if col_name == copy_raw_training_data.columns[col]] # store the imputed values in the dictionary

# Display performance metrics
print("Numerical variables performance for imputation 1:")
print(calc_num_metrics(actual_entries_store, imputed1_entries_store, num_cols))
print("\nCategorical variables performance for imputation 1:")
print(calc_categorical_metrics(actual_entries_store, imputed1_entries_store, categorical_cols))

Numerical variables performance for imputation 1:
     Variable     RMSE
       N_Days 1178.102
    Bilirubin    4.242
  Prothrombin    0.796
          Age 3782.324
  Cholesterol  129.944
      Albumin    0.349
       Copper   50.941
     Alk_Phos 2026.417
    Platelets   83.103
         SGOT   42.677
Tryglicerides   76.562
        Stage    0.897

Categorical variables performance for imputation 1:
    Variable  Accuracy  F1 Score
        Drug     0.570     0.414
         Sex     0.920     0.882
     Ascites     0.949     0.925
Hepatomegaly     0.549     0.389
     Spiders     0.809     0.723
       Edema     0.918     0.878


In [ ]:
# Imputation 2: for numerical variables, implement linear regression using the non-null rows to determine the
# regression lines. For rows with more 2 or 3 missing values, fill in 1 or 2, respecitvely, entries with the median for that column
np.random.seed(24)
# Store the missing data frame for only numerical variables. Store another data frame by dropping the null rows.
num_missing_raw_training_data = missing_raw_training_data.drop(categorical_cols, axis=1).drop(['Status'], axis=1)
linear_regres_df = num_missing_raw_training_data.dropna()

# fit linear regression models (for each numerical variable as the response) and store the coefficients and intercepts
regressions = {col: {'intercept': LinearRegression().fit(linear_regres_df.drop(col, axis=1),linear_regres_df[col]).intercept_,
                     'coeffs': LinearRegression().fit(linear_regres_df.drop(col, axis=1),linear_regres_df[col]).coef_}
               for col in linear_regres_df.columns}

# identify the rows with 1 missing value, and more than 1 missing value
rows_with_one_missing = num_missing_raw_training_data[num_missing_raw_training_data.isnull().sum(axis=1) == 1]
rows_with_over1_missing = num_missing_raw_training_data[num_missing_raw_training_data.isnull().sum(axis=1) >= 2]

# impute missing values using the median of the column in the rows with 2 or more missing values
median_vals = missing_raw_training_data.median() # store median value for each column in missing data frame
for idx, row in rows_with_over1_missing.iterrows(): # loop over rows with more than 1 missing value:
    missing_indices = row.isnull() # store the columns in current row with null entries
    fill_indices = np.random.choice(missing_indices[missing_indices].index, missing_indices.sum() - 1, replace=False) # sellect indicies to fill at random
    rows_with_over1_missing.loc[idx, fill_indices] = median_vals[fill_indices] #fill these entries using median of the column

# combine data frames such that each row only contains 1 missing value
new_rows_with_one_missing = pd.concat([rows_with_over1_missing, rows_with_one_missing]).sort_index()
#--------------------------------------------------------------------------------------------------------------------------
# impute missing values using linear regression
for idx, row in new_rows_with_one_missing.iterrows(): # loop over each row in this new data frame
    for col in num_cols: # loop over numerical variables:
        if pd.isna(row[col]): # if value in current cell is na:
            row[col] = (regressions[col]['coeffs'] * row.dropna()).sum() + regressions[col]['intercept'] # calc the imputed value using linear regression coefficients and intercept

# map to convert numerical variables in 'null_entries' to their names
mapping = {0: 'N_Days', 2: 'Age', 8: 'Bilirubin', 9: 'Cholesterol', 10: 'Albumin', 11: 'Copper', 12: 'Alk_Phos', 13: 'SGOT', 14: 'Tryglicerides', 15: 'Platelets', 16: 'Prothrombin', 17: 'Stage'}

# create dictionary for imputed values to be used in 'calc_num_metrics' function
imputed2_entries_store = {col_name: [] for col_name in mapping.values()} # initialise dictionary with values being empty list
for row, col in null_entries: # loop over each null entry:
    if col in mapping and isinstance(mapping[col], str): # if column variable is a key in mapping and value is a string:
        imputed2_entries_store[mapping[col]].append(new_rows_with_one_missing.loc[row, mapping[col]]) # add the imputed value to the corresponding list
#---------------------------------------------------------------------------------------------------------------------------
print("Numerical variables performance for imputation 2:")
print(calc_num_metrics(actual_entries_store, imputed2_entries_store, num_cols))

TypeError: Cannot convert [['D-penicillamine' 'Placebo' 'Placebo' ... 'D-penicillamine'
  'D-penicillamine' 'D-penicillamine']
 ['M' 'F' 'F' ... 'F' 'M' 'F']
 ['N' 'N' 'N' ... 'N' 'N' 'N']
 ...
 ['N' 'N' 'Y' ... 'Y' 'N' 'N']
 ['N' 'N' 'Y' ... 'S' 'N' 'N']
 ['D' 'C' 'D' ... 'D' 'D' 'C']] to numeric